Reprote a la Union por falta de catastro en grupos cañeros

In [26]:
import pandas as pd
import collections

from docxtpl import DocxTemplate

In [27]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [28]:
ruta

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [29]:
path_xlsx_avance = r'G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/AVANCE DE COSECHA V2.xlsx'

In [30]:
xlsx_entregas = pd.read_excel(path_xlsx_avance, sheet_name='ENTREGAS')
xlsx_avance = pd.read_excel(path_xlsx_avance, sheet_name = 'AVANCE POR GRUPO')

C:\Users\Usuario\anaconda3\envs\utea_reportes\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\Usuario\anaconda3\envs\utea_reportes\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [31]:
xlsx_entregas.head(3)

,INST,GRUPO,CODIGO CAÑERO,NOMBRE CAÑERO,COMPROMISO DE ENTREGA,PAQ,TM. ENTREGADAS,AVANCE,INICIO DE ENTREGAS,ULTIMA ENTREGA,COD_COSECHA
0,131,3,17,ACSAMA SEGOVIA SABINO,565.19,30.0,541.02,0.957236,45136.0,45139.0,356
1,126,1,18,ACUNA ARISPE HONORATO,517.69,8.0,128.12,0.247484,45138.0,45144.0,354
2,59,7,39,AGUILAR ROMERO NICOLAS,1020.46,69.0,752.04,0.736962,45080.0,45119.0,32


In [32]:
xlsx_avance.head(3)

,COD_COS,COMPROMISO,ENTREGA,_ENTREGA,TOTAL_AREA,COSECHA,SIN COSECHA,%COSECHA,TCH,DIF.AVANCE,AREA_ESTIM.,TCH_ESTIM,PRODUCCION_ESTIM,TN_COSECHA_ESTIM,DIF.ESTIM,% DIF.ESTIM,TN_EN_CAMPO,COMPROMISO_RESTANTE,DIFERENCIA
0,1,26449.77,19161.65,0.724454,543.38,304.38,239.00,0.560160,62.953052,0.164294,543.960674,55.385149,30127.342884,16876.146761,2285.503239,0.880725,13251.196123,7288.12,5963.076123
1,2,10848.49,5689.97,0.524494,170.45,60.19,110.26,0.353124,94.533477,0.171370,170.249063,50.971072,8677.777217,3064.332125,2625.637875,0.538550,5613.445092,5158.52,454.925092
2,3,5390.78,2792.22,0.517962,112.86,40.34,72.52,0.357434,69.217154,0.160528,114.920802,47.592543,5469.373160,1954.939866,837.280134,0.700138,3514.433294,2598.56,915.873294


## Argumentos

In [35]:
fecha_inf = '14 de agosto de 2023'
fecha_name = '14-08-2023'

fecha_img = '05 de agosto'
cod_grupo = 30
cod_grupos = [435, 190]

In [36]:
for i in cod_grupos:
    
    # filtrar el grupo
    grupo = xlsx_entregas[xlsx_entregas['COD_COSECHA'] == i].copy()
    grupo = grupo[['INST', 'CODIGO CAÑERO ', 'NOMBRE CAÑERO','COMPROMISO DE ENTREGA', 'TM. ENTREGADAS']]
    grupo['TM. ENTREGADAS'] = grupo['TM. ENTREGADAS'].fillna(0)
    grupo['avance%'] = grupo['TM. ENTREGADAS'] / grupo['COMPROMISO DE ENTREGA'] * 100
    grupo['compromiso_rest'] = grupo['COMPROMISO DE ENTREGA'] - grupo['TM. ENTREGADAS']
    
    # convertir en dataframe en lista de objetos
    list_obj_caneros = []
    for index, row in grupo.iterrows():
        canero = {
            'inst' : row['INST'],
            'cod_ca' : row['CODIGO CAÑERO '],
            'nom_ca' : row['NOMBRE CAÑERO'],
            'compromiso' : row['COMPROMISO DE ENTREGA'],
            'entrega' : row['TM. ENTREGADAS'],
            'entrega_porcen' : row['avance%'],
            'compromiso_rest' : row['compromiso_rest']
        }
        list_obj_caneros.append(collections.namedtuple("canero", canero.keys())(*canero.values()))
    
    # totales del grupo
    grupo_totales = {
        'compromiso' : grupo['COMPROMISO DE ENTREGA'].sum(),
        'entrega' : grupo['TM. ENTREGADAS'].sum(),
        'entrega_porcen' : grupo['TM. ENTREGADAS'].sum() / grupo['COMPROMISO DE ENTREGA'].sum() * 100,
        'compromiso_rest' : grupo['compromiso_rest'].sum()
    }
    obj_grupo_totales = collections.namedtuple("grupo_totales", grupo_totales.keys())(*grupo_totales.values())
    
    # filtrar datos de avance
    avance = xlsx_avance[xlsx_avance['COD_COS'] == i]
    
    # totales de avance
    avance_totales = {
        'fecha_img' : fecha_img,
        'fecha_informe' : fecha_inf,
        'ha_campo' : float(avance['SIN COSECHA']),
        'tn_campo' : float(avance['TN_EN_CAMPO']),
        'tch_estimado' : float(avance['TN_EN_CAMPO']) / float(avance['SIN COSECHA']),
        'diferencia' : float(avance['DIFERENCIA'])
    }
    
    # converitir datos totales de avance a objeto
    obj_avance_totales = collections.namedtuple("avance_totales",avance_totales.keys())(*avance_totales.values())
    
    doc = DocxTemplate(ruta + '/templates/tpl_rpt_avance_falta_catastro.docx')
    context = {
        'grupo' : list_obj_caneros,
        'grupo_totales' : obj_grupo_totales,
        'avance' : obj_avance_totales
    }
    doc.render(context)
    # CCC - consulta de caña en campo
    doc.save(ruta + '/informes/_' + str(i) + '_CCC_' + fecha_name + '.docx')
    print('Se generó reporte Num:', i)

Se generó reporte Num: 435
Se generó reporte Num: 190


## Datos del grupo

In [11]:
grupo = xlsx_entregas[xlsx_entregas['COD_COSECHA'] == cod_grupo].copy()
grupo = grupo[['INST', 'CODIGO CAÑERO ', 'NOMBRE CAÑERO','COMPROMISO DE ENTREGA', 'TM. ENTREGADAS']]
grupo['TM. ENTREGADAS'] = grupo['TM. ENTREGADAS'].fillna(0)
grupo['avance%'] = grupo['TM. ENTREGADAS'] / grupo['COMPROMISO DE ENTREGA'] * 100
grupo['compromiso_rest'] = grupo['COMPROMISO DE ENTREGA'] - grupo['TM. ENTREGADAS']
grupo

,INST,CODIGO CAÑERO,NOMBRE CAÑERO,COMPROMISO DE ENTREGA,TM. ENTREGADAS,avance%,compromiso_rest
6,50,75,AGUILERA TARADELLES NELSON MARIANO,26992.97,10171.47,37.681922,16821.50
48,50,577,AGUILERA MARIA INGRID WENDE DE,13433.25,13646.16,101.584948,-212.91
103,50,794,AGUILERA WENDE MARIANO,18084.70,18219.59,100.745879,-134.89
561,50,8005,AGUILERA WENDE MARIA INGRID,5201.02,54.64,1.050563,5146.38
1151,50,41715,VACA DIEZ UGARTE NICOLAS,1781.38,0.00,0.000000,1781.38


In [12]:
list_obj_caneros = []
for index, row in grupo.iterrows():
    canero = {
        'inst' : row['INST'],
        'cod_ca' : row['CODIGO CAÑERO '],
        'nom_ca' : row['NOMBRE CAÑERO'],
        'compromiso' : row['COMPROMISO DE ENTREGA'],
        'entrega' : row['TM. ENTREGADAS'],
        'entrega_porcen' : row['avance%'],
        'compromiso_rest' : row['compromiso_rest']
    }
    list_obj_caneros.append(collections.namedtuple("canero", canero.keys())(*canero.values()))
list_obj_caneros

[canero(inst=50, cod_ca=75, nom_ca='AGUILERA TARADELLES NELSON MARIANO ', compromiso=26992.97, entrega=10171.47, entrega_porcen=37.68192236719412, compromiso_rest=16821.5),
 canero(inst=50, cod_ca=577, nom_ca='AGUILERA MARIA INGRID WENDE DE', compromiso=13433.25, entrega=13646.16, entrega_porcen=101.58494779744291, compromiso_rest=-212.90999999999985),
 canero(inst=50, cod_ca=794, nom_ca='AGUILERA WENDE MARIANO', compromiso=18084.7, entrega=18219.59, entrega_porcen=100.7458791132836, compromiso_rest=-134.88999999999942),
 canero(inst=50, cod_ca=8005, nom_ca='AGUILERA WENDE MARIA INGRID ', compromiso=5201.02, entrega=54.64, entrega_porcen=1.0505631587650115, compromiso_rest=5146.38),
 canero(inst=50, cod_ca=41715, nom_ca='VACA DIEZ UGARTE NICOLAS', compromiso=1781.38, entrega=0.0, entrega_porcen=0.0, compromiso_rest=1781.38)]

## Totales del grupo

In [13]:
grupo_totales = {
    'compromiso' : grupo['COMPROMISO DE ENTREGA'].sum(),
    'entrega' : grupo['TM. ENTREGADAS'].sum(),
    'entrega_porcen' : grupo['TM. ENTREGADAS'].sum() / grupo['COMPROMISO DE ENTREGA'].sum() * 100,
    'compromiso_rest' : grupo['compromiso_rest'].sum()
}
obj_grupo_totales = collections.namedtuple("grupo_totales", grupo_totales.keys())(*grupo_totales.values())
obj_grupo_totales

grupo_totales(compromiso=65493.32, entrega=42091.86, entrega_porcen=64.26893612966941, compromiso_rest=23401.460000000003)

## Datos de avance

In [14]:
xlsx_avance.head(3)

,COD_COS,COMPROMISO,ENTREGA,_ENTREGA,TOTAL_AREA,COSECHA,SIN COSECHA,%COSECHA,TCH,DIF.AVANCE,AREA_ESTIM.,TCH_ESTIM,PRODUCCION_ESTIM,TN_COSECHA_ESTIM,DIF.ESTIM,% DIF.ESTIM,TN_EN_CAMPO,COMPROMISO_RESTANTE,DIFERENCIA
0,1,26449.77,19161.65,0.724454,543.38,304.38,239.00,0.560160,62.953052,0.164294,543.960674,55.385149,30127.342884,16876.146761,2285.503239,0.880725,13251.196123,7288.12,5963.076123
1,2,10848.49,5689.97,0.524494,170.45,60.19,110.26,0.353124,94.533477,0.171370,170.249063,50.971072,8677.777217,3064.332125,2625.637875,0.538550,5613.445092,5158.52,454.925092
2,3,5390.78,2792.22,0.517962,112.86,40.34,72.52,0.357434,69.217154,0.160528,114.920802,47.592543,5469.373160,1954.939866,837.280134,0.700138,3514.433294,2598.56,915.873294


In [15]:
avance = xlsx_avance[xlsx_avance['COD_COS'] == cod_grupo]
avance

,COD_COS,COMPROMISO,ENTREGA,_ENTREGA,TOTAL_AREA,COSECHA,SIN COSECHA,%COSECHA,TCH,DIF.AVANCE,AREA_ESTIM.,TCH_ESTIM,PRODUCCION_ESTIM,TN_COSECHA_ESTIM,DIF.ESTIM,% DIF.ESTIM,TN_EN_CAMPO,COMPROMISO_RESTANTE,DIFERENCIA
28,30,65493.32,42091.86,0.642689,856.71,452.11,404.6,0.527728,93.100927,0.114961,857.018376,76.385829,65464.059007,34547.228021,7544.631979,0.820758,30916.830986,23401.46,7515.370986


In [16]:
avance_totales = {
    'fecha_img' : fecha_img,
    'fecha_informe' : fecha_inf,
    'ha_campo' : float(avance['SIN COSECHA']),
    'tn_campo' : float(avance['TN_EN_CAMPO']),
    'tch_estimado' : float(avance['TN_EN_CAMPO']) / float(avance['SIN COSECHA']),
    'diferencia' : float(avance['DIFERENCIA'])
}
avance_totales

{'fecha_img': '05 de agosto',
 'fecha_informe': '14 de agosto de 2023',
 'ha_campo': 404.6,
 'tn_campo': 30916.830986172485,
 'tch_estimado': 76.41332423670906,
 'diferencia': 7515.370986172486}

In [17]:
obj_avance_totales = collections.namedtuple("avance_totales",avance_totales.keys())(*avance_totales.values())
obj_avance_totales

avance_totales(fecha_img='05 de agosto', fecha_informe='14 de agosto de 2023', ha_campo=404.6, tn_campo=30916.830986172485, tch_estimado=76.41332423670906, diferencia=7515.370986172486)

In [18]:
doc = DocxTemplate(ruta + '/templates/tpl_rpt_avance_falta_catastro.docx')
context = {
    'grupo' : list_obj_caneros,
    'grupo_totales' : obj_grupo_totales,
    'avance' : obj_avance_totales
}
doc.render(context)
doc.save(ruta + '/informes/rpt.docx')